In [31]:
from elftools.elf.elffile import ELFFile
import frida
import sys
from time import sleep

In [32]:
def generate_function_list(binary):
    """
    Generate a list of all the functions defined by the target executable.
    """
    functions = []
    with open(binary, "rb") as f:
        elf = ELFFile(f)
        for section in elf.iter_sections():
            if section.header.sh_type == 'SHT_SYMTAB':
                for symbol in section.iter_symbols():
                    if symbol.entry.st_info.type == 'STT_FUNC':
                        functions.append(symbol.name)
    return functions

In [33]:
binary_path = "./test/test_char"
args = ["arg1", "arg2"]
len=16

function_list = generate_function_list(binary_path)
print(function_list)

['deregister_tm_clones', 'register_tm_clones', '__do_global_dtors_aux', 'frame_dummy', '__libc_start_main@GLIBC_2.34', '_fini', 'strlen@GLIBC_2.2.5', 'f', 'printf@GLIBC_2.2.5', 'strcmp@GLIBC_2.2.5', '_start', 'g', 'main', '__cxa_finalize@GLIBC_2.2.5', '_init']


In [34]:
def make_script(f,n,len):
    args_str = ', '.join(f'args[{i}]' for i in range(n))
    return """
            Interceptor.attach(DebugSymbol.getFunctionByName('main'), {
                onEnter: function (args) {
                    send({function: 'main', args:args[1].readPointer().add(17).readCString()})
                },
                onLeave: function (retval) {
                	send({function: 'main', ret: retval})
                }
            });
        """

In [37]:
entries = []

def on_message(message, data):
    print(message)
    if message["type"] == "send" and message["payload"] != "done":
        function_name = message["payload"]["function"]
        try:
            function_args = message["payload"]["args"]
            io="input"
        except:
            function_args=message["payload"]["ret"]
            io="output"
        entries.append((function_name,function_args))
        print(entries[0][1])

# Run the binary
process = frida.spawn(binary_path, argv=[binary_path]+args)

sleep(1)

session = frida.attach(process)

function_list=['strlen', 'strcmp']
n=[1, 1, 0, 1, 1, 2]

script_txt=""
#for f,i in zip(function_list,n):
script_txt+= make_script('main',1,len)
script_txt+="\n"
    
script = session.create_script(script_txt)
script.on("message",on_message)
script.load()

frida.resume(process)

    # Wait for the script to complete
    #script.join()


{'type': 'send', 'payload': {'function': 'main', 'args': 'arg1'}}
arg1
{'type': 'send', 'payload': {'function': 'main', 'ret': '0x0'}}
arg1


0
